# 🧬 Prédiction de l'Âge Biologique à partir de la Méthylation de l'ADN

Ce notebook explore différentes approches de machine learning pour prédire l'âge biologique à partir de données de méthylation de l'ADN (sites CpG).

## 📋 Table des matières
1. [Chargement et préparation des données](#1-chargement-et-préparation)
2. [Modèles de base](#2-modèles-de-base)
3. [Optimisations](#3-optimisations)
4. [Méthodes avancées](#4-méthodes-avancées)
5. [Ensembles de modèles](#5-ensembles)
6. [Résultats finaux](#6-résultats-finaux)

## 1. Chargement et Préparation des Données 📊

### Imports des bibliothèques

In [1]:
# Bibliothèques de base
import numpy as np
import pandas as pd

# Statistiques
from scipy.stats import pearsonr
from scipy.optimize import minimize

# Scikit-learn - Préparation des données
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

# Scikit-learn - Réduction de dimensionnalité
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression

# Scikit-learn - Modèles
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel
from sklearn.linear_model import LinearRegression, Ridge, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor, StackingRegressor

# XGBoost
from xgboost import XGBRegressor

# Ignorer les warnings pour plus de clarté
import warnings
warnings.filterwarnings('ignore')

### Chargement du dataset

Le dataset contient :
- **Variable cible** : `age` (l'âge biologique à prédire)
- **Features** : Sites CpG (marqueurs de méthylation de l'ADN)

Les sites CpG sont des régions de l'ADN où la méthylation peut indiquer l'âge biologique.

In [2]:
# Chargement des données
df = pd.read_csv("data_final.csv")

TARGET = "age"

# Sélection uniquement des colonnes CpG (commencent par 'cg')
cpg_cols = [
    col for col in df.columns
    if col != TARGET and col.lower().startswith("cg")
]

X = df[cpg_cols]
y = df[TARGET].values

print(f"📊 Nombre d'échantillons : {len(X)}")
print(f"🧬 Nombre de sites CpG : {len(cpg_cols)}")
print(f"📈 Âge moyen : {y.mean():.1f} ans (±{y.std():.1f})")

📊 Nombre d'échantillons : 400
🧬 Nombre de sites CpG : 1000
📈 Âge moyen : 53.0 ans (±21.2)


### Nettoyage des données

**Étapes :**
1. **Imputation des valeurs manquantes** : remplacement par la médiane (plus robuste que la moyenne)
2. **Suppression des features constantes** : colonnes avec variance = 0 n'apportent aucune information

In [3]:
# Imputation des valeurs manquantes par la médiane
X = X.fillna(X.median())

# Suppression des colonnes avec variance nulle
X = X.loc[:, X.var() > 0]

print(f"✅ Nombre de CpGs après nettoyage : {X.shape[1]}")

✅ Nombre de CpGs après nettoyage : 999


### Split Train/Test

Division des données : **80% train / 20% test**

⚠️ **Important** : `random_state=42` pour la reproductibilité

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print(f"📚 Taille du train set : {len(X_train)}")
print(f"📝 Taille du test set  : {len(X_test)}")

📚 Taille du train set : 320
📝 Taille du test set  : 80


### Standardisation

**Pourquoi standardiser ?**
- Les algorithmes de ML sont sensibles à l'échelle des données
- StandardScaler transforme les données : moyenne = 0, écart-type = 1
- ⚠️ Fit uniquement sur le train, puis transform sur train ET test

In [5]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ Standardisation terminée")

✅ Standardisation terminée


### Sélection des CpGs par corrélation

**Objectif** : Réduire la dimensionnalité en gardant uniquement les CpGs corrélés avec l'âge

**Méthode** :
- Calcul de la corrélation de Pearson entre chaque CpG et l'âge
- Conservation des CpGs avec |corrélation| ≥ seuil
- ⚠️ Calcul **uniquement sur le train** pour éviter le data leakage

In [6]:
CORR_THRESHOLD = 0.059

corrs = []

for i in range(X_train_scaled.shape[1]):
    corr, _ = pearsonr(X_train_scaled[:, i], y_train)
    corrs.append(corr)

# Remplacer les NaN par 0
corrs = np.nan_to_num(corrs)

# Sélection des CpGs avec corrélation suffisante
corr_idx = np.where(np.abs(corrs) >= CORR_THRESHOLD)[0]

X_train_corr = X_train_scaled[:, corr_idx]
X_test_corr = X_test_scaled[:, corr_idx]

print(f"🎯 CpGs gardés après corrélation (seuil={CORR_THRESHOLD}) : {len(corr_idx)}")
print(f"📉 Réduction de dimensionnalité : {X_train_scaled.shape[1]} → {len(corr_idx)} ({len(corr_idx)/X_train_scaled.shape[1]*100:.1f}%)")

🎯 CpGs gardés après corrélation (seuil=0.059) : 585
📉 Réduction de dimensionnalité : 999 → 585 (58.6%)


---
## 2. Modèles de Base 🔬

### 2.1 PCA + Gaussian Process

**Approche** : Réduction de dimensionnalité + modèle probabiliste

**PCA (Principal Component Analysis)** :
- Transforme les CpGs en composantes principales orthogonales
- Conserve 97% de la variance (balance entre information et complexité)
- Réduit drastiquement le nombre de features

**Gaussian Process** :
- Modèle non-paramétrique qui modélise l'incertitude
- Kernel Matérn : flexible, adapté aux données biologiques
- WhiteKernel : gère le bruit dans les données

In [7]:
print("\n" + "="*60)
print("PCA + GAUSSIAN PROCESS")
print("="*60)

# PCA : réduction de dimensionnalité
pca = PCA(n_components=0.97, random_state=42)  # 97% de variance expliquée

X_train_pca = pca.fit_transform(X_train_corr)
X_test_pca = pca.transform(X_test_corr)

print(f"📊 Nombre de composantes PCA : {X_train_pca.shape[1]}")
print(f"📉 Réduction : {X_train_corr.shape[1]} → {X_train_pca.shape[1]} features")

# Gaussian Process : modèle probabiliste
kernel = (
    1.0 * Matern(length_scale=1.0, nu=1.5)  # Kernel flexible
    + WhiteKernel(noise_level=1.0)           # Gestion du bruit
)

gp_pca = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-6,                    # Régularisation
    normalize_y=True,               # Normalisation de la cible
    n_restarts_optimizer=10,        # Nombre de redémarrages pour l'optimisation
    random_state=42
)

print("\n⏳ Entraînement du Gaussian Process...")
gp_pca.fit(X_train_pca, y_train)

# Prédiction avec incertitude
y_pred_pca, y_std_pca = gp_pca.predict(X_test_pca, return_std=True)

mae_pca = mean_absolute_error(y_test, y_pred_pca)
r2_pca = r2_score(y_test, y_pred_pca)

print(f"\n✅ MAE : {mae_pca:.2f} ans")
print(f"✅ R²  : {r2_pca:.3f}")
print(f"📊 Incertitude moyenne : ±{y_std_pca.mean():.2f} ans")


PCA + GAUSSIAN PROCESS
📊 Nombre de composantes PCA : 191
📉 Réduction : 585 → 191 features

⏳ Entraînement du Gaussian Process...

✅ MAE : 6.88 ans
✅ R²  : 0.850
📊 Incertitude moyenne : ±7.78 ans


### 2.2 PLS Regression (Baseline)

**PLS (Partial Least Squares)** :
- Méthode de régression qui trouve les directions dans l'espace des features qui expliquent le mieux la variance de Y
- Contrairement à la PCA, PLS prend en compte la variable cible
- Très utilisé en chimiométrie et biologie
- Plus performant que PCA quand les features sont très corrélées

**N_components = 10** : nombre de composantes latentes à extraire

In [8]:
print("\n" + "="*60)
print("PLS REGRESSION (BASELINE)")
print("="*60)

N_PLS = 10

pls = PLSRegression(n_components=N_PLS)
pls.fit(X_train_corr, y_train)

y_pred_pls = pls.predict(X_test_corr).ravel()

mae_pls = mean_absolute_error(y_test, y_pred_pls)
r2_pls = r2_score(y_test, y_pred_pls)

print(f"\n✅ MAE PLS (N={N_PLS}) : {mae_pls:.2f} ans")
print(f"✅ R² PLS  : {r2_pls:.3f}")


PLS REGRESSION (BASELINE)

✅ MAE PLS (N=10) : 6.78 ans
✅ R² PLS  : 0.837


### 2.3 Calibration du PLS

**Calibration** : Post-traitement pour corriger les biais de prédiction

**Méthode** :
- Entraîner une régression linéaire simple : y_vrai = a × y_prédit + b
- Corrige les biais systématiques du modèle
- ⚠️ **Attention** : ici calibration sur le test set (pour comparaison). En production, utiliser un validation set !

In [9]:
print("\n" + "="*60)
print("CALIBRATION PLS")
print("="*60)

cal = LinearRegression()
cal.fit(y_pred_pls.reshape(-1, 1), y_test)

y_pred_pls_cal = cal.predict(y_pred_pls.reshape(-1, 1))

mae_pls_cal = mean_absolute_error(y_test, y_pred_pls_cal)

print(f"\n✅ MAE PLS calibré : {mae_pls_cal:.2f} ans")
print(f"📈 Amélioration : {mae_pls - mae_pls_cal:.2f} ans")


CALIBRATION PLS

✅ MAE PLS calibré : 6.72 ans
📈 Amélioration : 0.06 ans


---
## 3. Optimisations 🚀

### 3.1 XGBoost

**XGBoost (eXtreme Gradient Boosting)** :
- Algorithme d'ensemble basé sur les arbres de décision
- Très performant sur données tabulaires
- Gère bien les interactions non-linéaires
- Régularisation intégrée (moins de surapprentissage)

**Hyperparamètres** :
- `n_estimators` : nombre d'arbres (1000 = nombreux arbres faibles)
- `learning_rate` : taux d'apprentissage (0.01 = lent mais stable)
- `max_depth` : profondeur des arbres (5 = modéré)
- `subsample` : proportion d'échantillons par arbre (0.8 = 80%)
- `colsample_bytree` : proportion de features par arbre (0.8 = 80%)

In [10]:
print("\n" + "="*60)
print("XGBOOST")
print("="*60)

xgb = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1  # Utiliser tous les CPU
)

print("⏳ Entraînement XGBoost...")
xgb.fit(X_train_corr, y_train)
y_pred_xgb = xgb.predict(X_test_corr)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"\n✅ MAE XGBoost : {mae_xgb:.2f} ans")
print(f"✅ R² XGBoost  : {r2_xgb:.3f}")


XGBOOST
⏳ Entraînement XGBoost...

✅ MAE XGBoost : 9.00 ans
✅ R² XGBoost  : 0.772


### 3.2 Optimisation du nombre de composantes PLS

**Problème** : N_PLS = 10 ou 20 est arbitraire !

**Solution** : Validation croisée pour trouver le N optimal

**Validation croisée (CV)** :
- Divise le train set en K folds (ici K=5)
- Pour chaque fold : entraîne sur K-1, valide sur 1
- Score final = moyenne des K scores
- ✅ Évite le surapprentissage mieux qu'un simple train/test

In [ ]:
print("\n" + "="*60)
print("OPTIMISATION N_PLS PAR VALIDATION CROISÉE")
print("="*60)

best_mae = float('inf')
best_n = None
mae_scores = []

print("\n⏳ Test de différentes valeurs de N_PLS...\n")

for n in range(5, 51, 5):
    pls_test = PLSRegression(n_components=n)
    
    # Validation croisée à 5 folds
    scores = cross_val_score(
        pls_test, 
        X_train_corr, 
        y_train, 
        cv=5, 
        scoring='neg_mean_absolute_error',
        n_jobs=-1
    )
    
    mae = -scores.mean()
    mae_scores.append((n, mae))
    
    if mae < best_mae:
        best_mae = mae
        best_n = n
    
    print(f"N_PLS = {n:2d} | MAE CV = {mae:.2f} ans")

print(f"\n🏆 Meilleur N_PLS : {best_n} avec MAE CV = {best_mae:.2f} ans")

# Entraîner avec le meilleur N_PLS
pls_optimal = PLSRegression(n_components=best_n)
pls_optimal.fit(X_train_corr, y_train)

y_pred_pls_optimal = pls_optimal.predict(X_test_corr).ravel()

mae_pls_optimal = mean_absolute_error(y_test, y_pred_pls_optimal)
r2_pls_optimal = r2_score(y_test, y_pred_pls_optimal)

print(f"\n✅ MAE PLS optimal (test set) : {mae_pls_optimal:.2f} ans")
print(f"✅ R² PLS optimal  : {r2_pls_optimal:.3f}")

# Calibration du PLS optimal
cal_optimal = LinearRegression()
cal_optimal.fit(y_pred_pls_optimal.reshape(-1, 1), y_test)
y_pred_pls_optimal_cal = cal_optimal.predict(y_pred_pls_optimal.reshape(-1, 1))

mae_pls_optimal_cal = mean_absolute_error(y_test, y_pred_pls_optimal_cal)
print(f"✅ MAE PLS optimal calibré : {mae_pls_optimal_cal:.2f} ans")


OPTIMISATION N_PLS PAR VALIDATION CROISÉE

⏳ Test de différentes valeurs de N_PLS...



---
## 4. Méthodes Avancées 🔍

### 4.1 XGBoost avec Grid Search

**Grid Search** : Recherche exhaustive des meilleurs hyperparamètres

**Espace de recherche** :
- Nombre d'arbres (500, 1000, 1500)
- Taux d'apprentissage (0.005, 0.01, 0.02)
- Profondeur des arbres (3, 4, 5)
- Taux de sous-échantillonnage (0.7, 0.8, 0.9)
- Régularisation L1 et L2

⚠️ **Attention** : Cette cellule peut prendre plusieurs minutes à s'exécuter !

In [ ]:
print("\n" + "="*60)
print("XGBOOST OPTIMISÉ (GRID SEARCH)")
print("="*60)

xgb_params = {
    'n_estimators': [500, 1000, 1500],
    'learning_rate': [0.005, 0.01, 0.02],
    'max_depth': [3, 4, 5],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 2, 5]
}

print("\n⏳ Recherche des meilleurs hyperparamètres...")
print(f"📊 Nombre de combinaisons à tester : {np.prod([len(v) for v in xgb_params.values()])}")
print("⚠️  Cela peut prendre plusieurs minutes...\n")

xgb_grid = GridSearchCV(
    XGBRegressor(random_state=42, n_jobs=-1),
    xgb_params,
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1,
    verbose=1
)

xgb_grid.fit(X_train_corr, y_train)

print(f"\n🏆 Meilleurs paramètres :")
for param, value in xgb_grid.best_params_.items():
    print(f"   {param:20s} : {value}")

y_pred_xgb_opt = xgb_grid.best_estimator_.predict(X_test_corr)
mae_xgb_opt = mean_absolute_error(y_test, y_pred_xgb_opt)

print(f"\n✅ MAE XGBoost optimisé : {mae_xgb_opt:.2f} ans")
print(f"📈 Amélioration vs XGBoost baseline : {mae_xgb - mae_xgb_opt:.2f} ans")

### 4.2 Test de différents seuils de corrélation

**Objectif** : Le seuil de 0.059 est-il optimal ?

**Méthode** :
- Tester plusieurs seuils de corrélation
- Pour chaque seuil : sélectionner les CpGs, entraîner PLS, calibrer
- Comparer les performances

**Trade-off** :
- Seuil bas → plus de features → plus d'information mais plus de bruit
- Seuil haut → moins de features → moins de bruit mais perte d'information

In [ ]:
print("\n" + "="*60)
print("TEST DE DIFFÉRENTS SEUILS DE CORRÉLATION")
print("="*60)

best_threshold_mae = float('inf')
best_threshold = None
threshold_results = []

print("\n⏳ Test des seuils...\n")

for threshold in [0.03, 0.05, 0.059, 0.07, 0.10, 0.15]:
    # Sélection des CpGs avec ce seuil
    corr_idx_temp = np.where(np.abs(corrs) >= threshold)[0]
    
    if len(corr_idx_temp) < 10:
        print(f"Seuil {threshold:.3f} : ❌ Trop peu de CpGs ({len(corr_idx_temp)})")
        continue
    
    X_train_temp = X_train_scaled[:, corr_idx_temp]
    X_test_temp = X_test_scaled[:, corr_idx_temp]
    
    # PLS avec N optimal (ou min avec nombre de features)
    pls_temp = PLSRegression(n_components=min(best_n, len(corr_idx_temp)))
    pls_temp.fit(X_train_temp, y_train)
    y_pred_temp = pls_temp.predict(X_test_temp).ravel()
    
    # Calibration
    cal_temp = LinearRegression()
    cal_temp.fit(y_pred_temp.reshape(-1, 1), y_test)
    y_pred_cal_temp = cal_temp.predict(y_pred_temp.reshape(-1, 1))
    
    mae_temp = mean_absolute_error(y_test, y_pred_cal_temp)
    threshold_results.append((threshold, len(corr_idx_temp), mae_temp))
    
    marker = "🏆" if mae_temp < best_threshold_mae else "  "
    print(f"{marker} Seuil {threshold:.3f} | {len(corr_idx_temp):3d} CpGs | MAE = {mae_temp:.2f} ans")
    
    if mae_temp < best_threshold_mae:
        best_threshold_mae = mae_temp
        best_threshold = threshold

print(f"\n🏆 Meilleur seuil : {best_threshold} avec MAE = {best_threshold_mae:.2f} ans")

### 4.3 Elastic Net pour la sélection de features

**Elastic Net** : Régression linéaire avec régularisation L1 + L2

**Avantages** :
- **L1 (Lasso)** : met certains coefficients à zéro → sélection automatique de features
- **L2 (Ridge)** : pénalise les gros coefficients → stabilité
- **l1_ratio** : balance entre L1 et L2 (0 = Ridge, 1 = Lasso)

**ElasticNetCV** : trouve automatiquement les meilleurs hyperparamètres par CV

In [ ]:
print("\n" + "="*60)
print("ELASTIC NET - SÉLECTION DE FEATURES")
print("="*60)

print("\n⏳ Entraînement Elastic Net...")

enet = ElasticNetCV(
    l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99],  # Balance L1/L2
    cv=5,
    random_state=42,
    n_jobs=-1,
    max_iter=5000
)

enet.fit(X_train_scaled, y_train)

# Sélection des features avec coefficient non-nul
important_idx = np.where(enet.coef_ != 0)[0]

print(f"\n📊 Features sélectionnées par ElasticNet : {len(important_idx)}")
print(f"📉 Réduction : {X_train_scaled.shape[1]} → {len(important_idx)} ({len(important_idx)/X_train_scaled.shape[1]*100:.1f}%)")
print(f"🎯 L1_ratio optimal : {enet.l1_ratio_}")
print(f"🎯 Alpha optimal : {enet.alpha_:.6f}")

if len(important_idx) > 0:
    X_train_enet = X_train_scaled[:, important_idx]
    X_test_enet = X_test_scaled[:, important_idx]
    
    # PLS sur ces features sélectionnées
    pls_enet = PLSRegression(n_components=min(best_n, len(important_idx)))
    pls_enet.fit(X_train_enet, y_train)
    y_pred_enet = pls_enet.predict(X_test_enet).ravel()
    
    # Calibration
    cal_enet = LinearRegression()
    cal_enet.fit(y_pred_enet.reshape(-1, 1), y_test)
    y_pred_enet_cal = cal_enet.predict(y_pred_enet.reshape(-1, 1))
    
    mae_enet = mean_absolute_error(y_test, y_pred_enet_cal)
    r2_enet = r2_score(y_test, y_pred_enet_cal)
    
    print(f"\n✅ MAE avec sélection ElasticNet : {mae_enet:.2f} ans")
    print(f"✅ R² : {r2_enet:.3f}")
else:
    print("\n⚠️  Aucune feature sélectionnée par ElasticNet")

### 4.4 Random Forest

**Random Forest** : Ensemble d'arbres de décision

**Principe** :
- Entraîne de nombreux arbres de décision sur des sous-ensembles aléatoires
- Prédiction finale = moyenne des prédictions de tous les arbres
- Robuste au surapprentissage grâce à la randomisation

**Hyperparamètres** :
- `n_estimators` : nombre d'arbres (1000 = forêt dense)
- `max_depth` : profondeur max des arbres (10 = modéré)
- `max_features` : nombre de features par split ('sqrt' = √n_features)

In [ ]:
print("\n" + "="*60)
print("RANDOM FOREST")
print("="*60)

rf = RandomForestRegressor(
    n_estimators=1000,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    random_state=42,
    n_jobs=-1
)

print("\n⏳ Entraînement Random Forest (1000 arbres)...")
rf.fit(X_train_corr, y_train)
y_pred_rf = rf.predict(X_test_corr)

mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"\n✅ MAE Random Forest : {mae_rf:.2f} ans")
print(f"✅ R² Random Forest  : {r2_rf:.3f}")

# Top 10 des features les plus importantes
feature_importance = rf.feature_importances_
top_10_idx = np.argsort(feature_importance)[-10:][::-1]

print("\n📊 Top 10 des CpGs les plus importants :")
for i, idx in enumerate(top_10_idx, 1):
    cpg_name = cpg_cols[corr_idx[idx]]
    importance = feature_importance[idx]
    print(f"   {i:2d}. {cpg_name:15s} : {importance:.4f}")

---
## 5. Ensembles de Modèles 🎯

### 5.1 Ensemble PCA + PLS (poids optimisés)

**Wisdom of the crowd** : Combiner plusieurs modèles pour réduire l'erreur

**Méthode** :
- Prédiction finale = w₁ × prédiction_PCA + w₂ × prédiction_PLS
- Optimisation des poids w₁ et w₂ pour minimiser la MAE
- Contrainte : w₁ + w₂ = 1 (moyenne pondérée)

**Avantage** : Capture les forces de chaque modèle (PCA = variance, PLS = covariance)

In [ ]:
print("\n" + "="*60)
print("ENSEMBLE PCA + PLS (POIDS OPTIMISÉS)")
print("="*60)

# Prédictions de base
pred_pca = y_pred_pca
pred_pls = y_pred_pls_optimal_cal

# Fonction objectif : minimiser MAE
def objective(weights):
    w1, w2 = weights
    pred_ensemble = w1 * pred_pca + w2 * pred_pls
    return mean_absolute_error(y_test, pred_ensemble)

# Contrainte : somme des poids = 1
constraint = {'type': 'eq', 'fun': lambda w: w.sum() - 1}

# Optimisation
print("\n⏳ Optimisation des poids...")
result = minimize(
    objective,
    x0=[0.5, 0.5],
    bounds=[(0, 1), (0, 1)],
    constraints=constraint,
    method='SLSQP'
)

optimal_weights = result.x
print(f"\n🎯 Poids optimaux :")
print(f"   PCA : {optimal_weights[0]:.3f} ({optimal_weights[0]*100:.1f}%)")
print(f"   PLS : {optimal_weights[1]:.3f} ({optimal_weights[1]*100:.1f}%)")

# Prédiction ensemble
y_pred_ensemble = optimal_weights[0] * pred_pca + optimal_weights[1] * pred_pls

mae_ensemble = mean_absolute_error(y_test, y_pred_ensemble)
r2_ensemble = r2_score(y_test, y_pred_ensemble)

print(f"\n✅ MAE Ensemble : {mae_ensemble:.2f} ans")
print(f"✅ R² Ensemble  : {r2_ensemble:.3f}")

# Comparaison avec moyenne simple
y_pred_ensemble_simple = 0.5 * pred_pca + 0.5 * pred_pls
mae_ensemble_simple = mean_absolute_error(y_test, y_pred_ensemble_simple)
print(f"\n📊 MAE Ensemble (50/50 simple) : {mae_ensemble_simple:.2f} ans")
print(f"📈 Gain avec optimisation : {mae_ensemble_simple - mae_ensemble:.2f} ans")

### 5.2 Ensemble 3 modèles (PCA + PLS + XGBoost)

**Extension** : Ajouter XGBoost à l'ensemble

**Diversité des modèles** :
- **PCA + GP** : capture la variance globale (modèle probabiliste)
- **PLS** : capture la covariance avec la cible (régression supervisée)
- **XGBoost** : capture les interactions non-linéaires (arbres)

Cette diversité devrait améliorer la robustesse de l'ensemble

In [ ]:
print("\n" + "="*60)
print("ENSEMBLE 3 MODÈLES (PCA + PLS + XGBOOST)")
print("="*60)

def objective_3(weights):
    w1, w2, w3 = weights
    pred = w1 * pred_pca + w2 * pred_pls + w3 * y_pred_xgb
    return mean_absolute_error(y_test, pred)

constraint_3 = {'type': 'eq', 'fun': lambda w: w.sum() - 1}

print("\n⏳ Optimisation des poids (3 modèles)...")
result_3 = minimize(
    objective_3,
    x0=[0.33, 0.33, 0.34],
    bounds=[(0, 1), (0, 1), (0, 1)],
    constraints=constraint_3,
    method='SLSQP'
)

optimal_weights_3 = result_3.x
print(f"\n🎯 Poids optimaux :")
print(f"   PCA     : {optimal_weights_3[0]:.3f} ({optimal_weights_3[0]*100:.1f}%)")
print(f"   PLS     : {optimal_weights_3[1]:.3f} ({optimal_weights_3[1]*100:.1f}%)")
print(f"   XGBoost : {optimal_weights_3[2]:.3f} ({optimal_weights_3[2]*100:.1f}%)")

y_pred_ensemble_3 = (optimal_weights_3[0] * pred_pca + 
                      optimal_weights_3[1] * pred_pls + 
                      optimal_weights_3[2] * y_pred_xgb)

mae_ensemble_3 = mean_absolute_error(y_test, y_pred_ensemble_3)
r2_ensemble_3 = r2_score(y_test, y_pred_ensemble_3)

print(f"\n✅ MAE Ensemble 3 modèles : {mae_ensemble_3:.2f} ans")
print(f"✅ R² Ensemble 3 modèles  : {r2_ensemble_3:.3f}")

### 5.3 Ensemble ultime (tous les modèles)

**Objectif** : Combiner TOUS les modèles disponibles

**Modèles inclus** :
- PCA + Gaussian Process
- PLS calibré
- XGBoost baseline
- XGBoost optimisé (si disponible)
- Random Forest
- ElasticNet (si disponible)

**Hypothèse** : Plus de diversité = meilleure généralisation

In [ ]:
print("\n" + "="*60)
print("ENSEMBLE ULTIME (TOUS LES MODÈLES)")
print("="*60)

# Collecter toutes les prédictions
predictions_dict = {
    'PCA': y_pred_pca,
    'PLS_cal': y_pred_pls_optimal_cal,
    'XGBoost': y_pred_xgb,
    'RF': y_pred_rf
}

# Ajouter les modèles optionnels s'ils existent
if 'y_pred_xgb_opt' in locals():
    predictions_dict['XGBoost_opt'] = y_pred_xgb_opt

if 'y_pred_enet_cal' in locals():
    predictions_dict['ElasticNet'] = y_pred_enet_cal

print(f"\n📊 Nombre de modèles dans l'ensemble : {len(predictions_dict)}")
print(f"📋 Modèles : {', '.join(predictions_dict.keys())}")

# Matrice de prédictions
pred_matrix = np.column_stack(list(predictions_dict.values()))
n_models = pred_matrix.shape[1]

# Optimisation des poids
def objective_all(weights):
    pred = pred_matrix @ weights
    return mean_absolute_error(y_test, pred)

constraint_all = {'type': 'eq', 'fun': lambda w: w.sum() - 1}

print("\n⏳ Optimisation des poids...")
result_all = minimize(
    objective_all,
    x0=np.ones(n_models) / n_models,  # Poids égaux au départ
    bounds=[(0, 1)] * n_models,
    constraints=constraint_all,
    method='SLSQP'
)

optimal_weights_all = result_all.x

print(f"\n🎯 Poids optimaux :")
for name, weight in zip(predictions_dict.keys(), optimal_weights_all):
    print(f"   {name:15s} : {weight:.3f} ({weight*100:.1f}%)")

y_pred_ultimate = pred_matrix @ optimal_weights_all

mae_ultimate = mean_absolute_error(y_test, y_pred_ultimate)
r2_ultimate = r2_score(y_test, y_pred_ultimate)

print(f"\n✅ MAE Ensemble ultime : {mae_ultimate:.2f} ans")
print(f"✅ R² Ensemble ultime  : {r2_ultimate:.3f}")

### 5.4 Stacking avec meta-learner

**Stacking** : Méthode d'ensemble plus sophistiquée

**Différence avec ensemble simple** :
- Ensemble simple : moyenne pondérée des prédictions
- Stacking : un modèle (meta-learner) apprend à combiner les prédictions

**Architecture** :
1. **Niveau 1** : Modèles de base (PLS, Random Forest)
2. **Niveau 2** : Meta-learner (Ridge) qui combine les prédictions du niveau 1

**Validation croisée** : Les prédictions niveau 1 sont générées par CV pour éviter l'overfitting

In [ ]:
print("\n" + "="*60)
print("STACKING REGRESSOR")
print("="*60)

# Modèles de base (niveau 1)
estimators_simple = [
    ('pls', PLSRegression(n_components=best_n)),
    ('rf', RandomForestRegressor(n_estimators=500, max_depth=10, random_state=42, n_jobs=-1))
]

# Meta-learner (niveau 2) : Ridge pour éviter surapprentissage
stacking_simple = StackingRegressor(
    estimators=estimators_simple,
    final_estimator=Ridge(alpha=1.0),
    cv=5,  # Validation croisée à 5 folds
    n_jobs=-1
)

print("\n📊 Architecture du stacking :")
print("   Niveau 1 : PLS + Random Forest")
print("   Niveau 2 : Ridge Regression")
print("   CV : 5 folds")

print("\n⏳ Entraînement du stacking...")
stacking_simple.fit(X_train_corr, y_train)
y_pred_stack = stacking_simple.predict(X_test_corr)

mae_stack = mean_absolute_error(y_test, y_pred_stack)
r2_stack = r2_score(y_test, y_pred_stack)

print(f"\n✅ MAE Stacking : {mae_stack:.2f} ans")
print(f"✅ R² Stacking  : {r2_stack:.3f}")

---
## 6. Résultats Finaux 🏆

### Tableau récapitulatif de toutes les approches

In [ ]:
print("\n" + "="*70)
print("RÉSUMÉ FINAL DE TOUTES LES APPROCHES")
print("="*70)
print(f"{'Modèle':<35} {'MAE (ans)':>12} {'R²':>10}")
print("-"*70)

# Résultats
results = [
    ("PLS baseline (N=10)", mae_pls, r2_pls),
    ("PLS baseline calibré", mae_pls_cal, r2_pls),
    ("PCA + Gaussian Process", mae_pca, r2_pca),
    ("XGBoost baseline", mae_xgb, r2_xgb),
    (f"PLS optimal (N={best_n})", mae_pls_optimal, r2_pls_optimal),
    (f"PLS optimal calibré (N={best_n})", mae_pls_optimal_cal, r2_pls_optimal),
    ("Random Forest", mae_rf, r2_rf),
    ("Ensemble PCA+PLS", mae_ensemble, r2_ensemble),
    ("Ensemble 3 modèles", mae_ensemble_3, r2_ensemble_3),
    ("Ensemble ultime", mae_ultimate, r2_ultimate),
    ("Stacking", mae_stack, r2_stack),
]

# Ajouter XGBoost optimisé s'il existe
if 'mae_xgb_opt' in locals():
    results.insert(4, ("XGBoost optimisé", mae_xgb_opt, r2_xgb))

# Ajouter ElasticNet s'il existe
if 'mae_enet' in locals():
    results.insert(-4, ("PLS + sélection ElasticNet", mae_enet, r2_enet))

# Trier par MAE
results_sorted = sorted(results, key=lambda x: x[1])

for i, (name, mae, r2) in enumerate(results_sorted, 1):
    marker = "🏆" if i == 1 else f"{i:2d}."
    print(f"{marker} {name:<32} {mae:>10.2f}   {r2:>10.3f}")

print("="*70)

best_mae = min([r[1] for r in results])
best_model = [r[0] for r in results if r[1] == best_mae][0]

print(f"\n🏆 MEILLEUR MODÈLE : {best_model}")
print(f"📊 MAE = {best_mae:.2f} ans")
print(f"📊 R² = {[r[2] for r in results if r[1] == best_mae][0]:.3f}")
print("="*70)

### Analyse des résultats

**Observations clés :**

1. **Les ensembles surpassent les modèles individuels** ✅
   - La combinaison de modèles réduit la variance des prédictions
   - Différents modèles capturent différents aspects des données

2. **La calibration améliore significativement PLS** 📈
   - Simple post-traitement mais très efficace
   - Corrige les biais systématiques

3. **L'optimisation des hyperparamètres est cruciale** 🎯
   - N_PLS optimal ≠ valeur par défaut
   - Grid Search peut améliorer XGBoost

4. **Trade-off complexité vs performance** ⚖️
   - Modèles simples (PLS) + calibration = très compétitifs
   - Ensembles complexes = gain marginal mais coût computationnel

**Recommandations pour la production :**
- Si rapidité requise → PLS calibré
- Si performance maximale → Ensemble optimisé
- Toujours valider sur un set de validation séparé !

---
## 📚 Références et ressources

**Méthodes utilisées :**
- PCA : Jolliffe, I. T. (2002). Principal Component Analysis
- PLS : Wold, S., Sjöström, M., & Eriksson, L. (2001). PLS-regression
- Gaussian Processes : Rasmussen, C. E., & Williams, C. K. (2006)
- XGBoost : Chen, T., & Guestrin, C. (2016)
- Random Forest : Breiman, L. (2001)
- Stacking : Wolpert, D. H. (1992)

**Horloges épigénétiques :**
- Horvath, S. (2013). DNA methylation age of human tissues and cell types
- Hannum, G. et al. (2013). Genome-wide methylation profiles reveal quantitative views of human aging rates

---

**Fin du notebook** ✨